# Day 14 - Extended Polymerization

## Part 1
Apply **10 steps of pair insertion** [...] What do you get if you take the quantity of the **most common element** and subtract the quantity of the **least common element**?

In [27]:
Polymer = list[str]
Rules = dict[str, str]

def parse(file) -> tuple[Polymer, Rules]:
    with open(file, "r") as f:
        lines = f.readlines()
    
    polymer = Polymer(element for element in lines[0].strip())
    
    rules = Rules()
    for line in lines[2:]:
        line = line.strip().replace(" ", "")
        if not line:
            continue
        
        sequence, insertion = line.split("->")
        rules[sequence] = insertion
    
    return polymer, rules

In [28]:
test_data, test_rules = parse("14_test.in")
data, rules = parse("14_input.in")

In [29]:
def pair_insertion(polymer: Polymer, rules: Rules, steps=10): 
    print(f"Template: {''.join(polymer):>16}")
    polymer = polymer.copy()
    for i in range(1, steps+1):
        new_polymer = polymer[0:1]  
        previous = polymer.pop(0)
        
        while polymer:
            current = polymer.pop(0)
            sequence = previous + current
            if sequence in rules:
                insertion = rules[sequence]
                new_polymer.append(insertion)
            new_polymer.append(current)
            previous = current
            
        # print(f"After Step {i:03d}:       {''.join(new_polymer)}")
        polymer = new_polymer

    return polymer
        

In [30]:
pair_insertion(test_data, test_rules, steps=3)

Template:             NNCB


['N',
 'B',
 'B',
 'B',
 'C',
 'N',
 'C',
 'C',
 'N',
 'B',
 'B',
 'N',
 'B',
 'N',
 'B',
 'B',
 'C',
 'H',
 'B',
 'H',
 'H',
 'B',
 'C',
 'H',
 'B']

In [31]:
def calculate_part1_score(polymer):
    elements = set(polymer)
    counts = {element: polymer.count(element)
              for element in elements}
    maximum = max(counts, key=lambda k: counts[k])
    minimum = min(counts, key=lambda k: counts[k])
    
    print(f"Max Element: {maximum} {counts[maximum]}")
    print(f"Min Element: {minimum} {counts[minimum]}")
    print(f"Result: {counts[maximum] - counts[minimum]}")
    

In [32]:
result = pair_insertion(test_data, test_rules, steps=10)
calculate_part1_score(result)

Template:             NNCB
Max Element: B 1749
Min Element: H 161
Result: 1588


In [33]:
result = pair_insertion(data, rules, steps=10)
calculate_part1_score(result)

Template: HBCHSNFFVOBNOFHFOBNO
Max Element: H 3978
Min Element: O 570
Result: 3408


## Part 2 - The same, but more and smart
Kinda like Day 06 Lanternfish

In [34]:
result = pair_insertion(test_data, test_rules, steps=15)
calculate_part1_score(result)

Template:             NNCB
Max Element: B 60184
Min Element: H 3292
Result: 56892


In [35]:
result = pair_insertion(test_data, test_rules, steps=16)
calculate_part1_score(result)

Template:             NNCB
Max Element: B 121971
Min Element: H 4951
Result: 117020


In [36]:
# Runtime: ~10s
# result = pair_insertion(test_data, test_rules, steps=17)
# calculate_part1_score(result)

In [37]:
# Runtime: ~40s
# result = pair_insertion(test_data, test_rules, steps=18)
# calculate_part1_score(result)

Idea
----
Use a register of adjacent elements.
From there, half the amount since we count each element twice (except for the start and end element).

In [49]:
from collections import defaultdict
import pprint

def smart_pair_insertion(polymer, rules, steps=10):
    start = polymer[0]
    end = polymer[-1]
    
    # Parse Polymer.
    adjacent_elements = defaultdict(int)
    for i in range(len(polymer) -1):
        sequence = "".join(polymer[i:i+2])
        adjacent_elements[sequence] += 1
    
    # Simulate pair insertion.
    for step in range(steps):
        new_adjacent_elements = defaultdict(int)
        for sequence, value in adjacent_elements.items():
            if sequence not in rules:
                # Keep "inert" sequences
                new_adjacent_elements[sequence] += value
                continue
            
            # Insert according to rules
            insert = rules[sequence]
            new_adjacent_elements[sequence[0] + insert] += value
            new_adjacent_elements[insert + sequence[1]] += value
            
        adjacent_elements = new_adjacent_elements
    pprint.pprint(dict(adjacent_elements))
    
    # Calculate occurrences.
    occurrences = defaultdict(int)
    for key, value in adjacent_elements.items():
        e1, e2 = key
        occurrences[e1] += value
        occurrences[e2] += value
    
    occurrences[start] -= 1
    occurrences[end] -= 1

    occurrences = {key: int(value/2) for key, value in occurrences.items()}

    occurrences[start] += 1
    occurrences[end] += 1
    
    pprint.pprint(occurrences)
    
    # Calculate score.
    score = max(occurrences.values()) - min(occurrences.values())
    print(f"Polymer Score: {score}")
        

In [50]:
smart_pair_insertion(test_data, test_rules, steps=40)

{'BB': 1094624367533,
 'BC': 2903343622,
 'BH': 1490759980,
 'BN': 1093021098466,
 'CB': 1554005966,
 'CC': 1451671811,
 'CH': 1050598772,
 'CN': 2541358752,
 'HB': 1388425825,
 'HC': 1152932927,
 'HH': 823172187,
 'HN': 485345134,
 'NB': 1094472770278,
 'NC': 1089686941,
 'NH': 485345134}
{'B': 2192039569602, 'C': 6597635301, 'H': 3849876073, 'N': 1096047802353}
Polymer Score: 2188189693529


In [51]:
smart_pair_insertion(data, rules, steps=40)

{'BB': 252039032349,
 'BC': 90880725358,
 'BH': 485722628631,
 'BK': 11362878492,
 'BN': 5682173332,
 'BO': 161443022690,
 'BP': 638768762454,
 'BS': 2215327848,
 'BV': 80731554266,
 'CF': 78980591623,
 'CH': 180150206467,
 'CN': 360205890666,
 'CP': 196734740492,
 'FB': 196687815500,
 'FC': 340761739370,
 'FF': 157980009113,
 'FH': 654461986969,
 'FK': 524391232760,
 'FN': 523516774575,
 'FS': 315991099959,
 'FV': 340234529329,
 'HB': 749934532670,
 'HC': 181751237498,
 'HF': 549267587088,
 'HH': 443800261626,
 'HK': 262165060722,
 'HN': 392498128810,
 'HO': 316135611349,
 'HP': 782712705394,
 'HV': 612067546434,
 'KB': 307457248868,
 'KC': 22723645313,
 'KH': 610920035370,
 'KP': 307509035465,
 'KV': 360820615920,
 'NB': 1107985134,
 'NC': 134510397622,
 'NF': 243206757746,
 'NH': 327222437055,
 'NK': 169288384808,
 'NN': 71314948461,
 'NP': 134147574441,
 'NS': 316043230321,
 'NV': 388915669231,
 'OB': 2215327848,
 'OF': 275172703368,
 'OH': 137637374905,
 'OK': 1034059386,
 'ON': 7